In [1]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Trainer, TrainingArguments
import torch
import os
import pandas as pd
from tqdm import tqdm
from datasets import Dataset
from fuzzywuzzy import fuzz
import evaluate
import csv
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import top_k_accuracy_score, ndcg_score
import random

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/mobilerec/splits/train.jsonl"
df_recommender_train = pd.read_json(input_file, lines=True)
for _, row in df_recommender_train.iterrows():
    row["recommended_app"]["app_name"] = row["recommended_app"]["app_name"].lower()

In [4]:
df_recommender_train

,user_id,user_previous_interactions,recommended_app,negative_recommended_app,turns
0,QpiHlryI4pvcIEjc,"[{'app_name': 'Threema', 'app_package': 'ch.th...",{'app_name': 'chug patrol kid train: ready to ...,[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,AlktS4aqUweiO6c2,"[{'app_name': 'Sector Strike', 'app_package': ...","{'app_name': 'cardboard design lab', 'package_...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,hNdOcoE3IfbPpQQv,[],"{'app_name': 'nasty goats', 'package_name': 'c...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,9OJ3TBx83hlcGRW3,[{'app_name': 'RE/MAX Real Estate Search App (...,"{'app_name': 'camscanner (license)', 'package_...","[{'app_name': 'CamScanner - PDF Scanner App', ...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,pgwkSwLUE5VPU9vQ,[],"{'app_name': 'lego ® batman: beyond gotham', '...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
8630,FVXHRzhqHTg3AfU8,"[{'app_name': 'BodyFast Intermittent Fasting',...","{'app_name': 'faceapp: face editor', 'package_...",[{'app_name': 'Photoshop Express Photo Editor'...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8631,15CoCtoYc3KXIdvy,"[{'app_name': 'Hidden Through Time', 'app_pack...","{'app_name': 'the walking dead: survivors', 'p...","[{'app_name': 'Plants vs. Zombies™ Heroes', 'p...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8632,OroJNW8u3I4LORPC,"[{'app_name': 'Weedmaps: Marijuana, Cannabis, ...","{'app_name': 'hay day', 'package_name': 'com.s...","[{'app_name': 'Farming Simulator 18', 'package...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
8633,h5nCG9WD0a6HYZYj,"[{'app_name': 'Wordament® by Microsoft', 'app_...","{'app_name': 'socratic by google', 'package_na...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [5]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/mobilerec/splits/val.jsonl"
df_recommender_validation = pd.read_json(input_file, lines=True)
for _, row in df_recommender_validation.iterrows():
    row["recommended_app"]["app_name"] = row["recommended_app"]["app_name"].lower()

In [6]:
df_recommender_validation

,user_id,user_previous_interactions,recommended_app,negative_recommended_app,turns
0,7oYEjVyJoTb1Spl3,"[{'app_name': 'DEAD TRIGGER 2: Zombie Games', ...",{'app_name': 'az screen recorder - video recor...,"[{'app_name': 'Mobizen Screen Recorder', 'pack...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1,qREr480MftU5cHmf,"[{'app_name': 'Walli - 4K Wallpapers', 'app_pa...","{'app_name': 'dfndr security: antivirus', 'pac...","[{'app_name': 'Google Authenticator', 'package...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
2,932Lp6L51OFUf7G4,"[{'app_name': 'Dynasty Origins: Pioneer', 'app...","{'app_name': 'onmyoji arena', 'package_name': ...","[{'app_name': 'Legend of Ace', 'package_name':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
3,lloZoTmrDQU1UT6h,"[{'app_name': 'Star Warfare2:Payback', 'app_pa...","{'app_name': 'good pizza, great pizza', 'packa...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
4,3Ry4S118042YaHsQ,"[{'app_name': 'Road to Valor: World War II', '...","{'app_name': 'pepi wonder world: magic isle!',...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
...,...,...,...,...,...
1842,f9btKO9JA3uTiNNT,"[{'app_name': 'Public.com - Stocks & Crypto', ...","{'app_name': 'mobile legends: bang bang', 'pac...","[{'app_name': 'Onmyoji Arena', 'package_name':...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1843,hTIn3sXfLCYgqFl2,"[{'app_name': 'Crew Messaging and Scheduling',...","{'app_name': 'tiles hop: edm rush!', 'package_...",[{'app_name': 'Piano Music Go-EDM Piano Games'...,"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1844,frAqBnUgTJEoFizS,"[{'app_name': 'Google Calendar', 'app_package'...","{'app_name': 'likee lite - let you shine', 'pa...","[{'app_name': 'TikTok', 'package_name': 'com.z...","[{'turn': 1, 'is_rec': False, 'user_accept_rec..."
1845,uhegyP6nZH3ZDh6D,"[{'app_name': 'Broken Dawn II HD', 'app_packag...","{'app_name': 'blockman go', 'package_name': 'c...",[],"[{'turn': 1, 'is_rec': False, 'user_accept_rec..."


In [7]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "google/flan-t5-base")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
prompt_train = []
recommend_train = []
not_founds = 0

for _, row in tqdm(df_recommender_train.iterrows(), total=len(df_recommender_train)):
    prompt = ""
    found = False
    recommended = row["recommended_app"]["app_name"] 
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
        
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_train.append(prompt)
                recommend_train.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"len(prompt_train): {len(prompt_train)}")
print(f"len(recommend_train): {len(recommend_train)}")

            
            
prompt_encodings = tokenizer(prompt_train, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_train, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_train = Dataset.from_dict(dataset)

100%|██████████| 8635/8635 [00:30<00:00, 278.87it/s]


Could not find 145
len(prompt_train): 8490
len(recommend_train): 8490


In [9]:
prompt_validation = []
recommend_validation = []
not_founds = 0

for _, row in tqdm(df_recommender_validation.iterrows(), total=len(df_recommender_validation)):
    prompt = ""
    found = False
    recommended = row["recommended_app"]["app_name"] 
    
    for index, turn in enumerate(row["turns"]):
        if "COMPUTER" in turn:
            computer = turn["COMPUTER"]
            
            if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
                prompt += "computer: I would recommend the "
                prompt_validation.append(prompt)
                recommend_validation.append(recommended)
                found = True
                break
            else:
                prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1
        
print(f"Could not find {not_founds}")
print(f"len(prompt_validation): {len(prompt_validation)}")
print(f"len(recommend_validation): {len(recommend_validation)}")
            
            
prompt_encodings = tokenizer(prompt_validation, padding='max_length', max_length=1024, truncation=True, return_tensors='pt')
recommend_encodings = tokenizer(recommend_validation, padding='max_length', max_length=32, truncation=True, return_tensors='pt')

labels = recommend_encodings['input_ids']
labels[labels == tokenizer.pad_token_id] = IGNORE_INDEX

dataset = {
    'input_ids': prompt_encodings['input_ids'],
    'attention_mask': prompt_encodings['attention_mask'],
    'labels': labels,
}
dataset_validation = Dataset.from_dict(dataset)

100%|██████████| 1847/1847 [00:06<00:00, 273.09it/s]


Could not find 24
len(prompt_validation): 1823
len(recommend_validation): 1823


In [10]:
def data_collator(batch):
    input_ids, attention_mask, labels,  = [], [], []
    for sample in batch:
        input_ids.append(sample['input_ids'])
        attention_mask.append(sample['attention_mask'])
        labels.append(sample['labels'])
    max_encoder_len = max(sum(x) for x in attention_mask)
    max_decoder_len = max(sum([0 if item == IGNORE_INDEX else 1 for item in x]) for x in labels)
    return {
        'input_ids': torch.tensor(input_ids)[:, :max_encoder_len],
        'attention_mask': torch.tensor(attention_mask)[:, :max_encoder_len],
        'labels': torch.tensor(labels)[:, :max_decoder_len]
    }

In [11]:
training_args = TrainingArguments(
    output_dir="/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/mobilerec/T5_recommender",
    num_train_epochs=5,
    # logging_steps=500,
    # logging_dir=self.cfg.logging_dir,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_strategy="steps",
    evaluation_strategy="steps",
    save_steps=0.3,#self.cfg.save_steps,
    eval_steps=0.3, #self.cfg.eval_steps,
    save_total_limit=3,
    gradient_accumulation_steps=3, #gradient_accumulation_steps,
    per_device_train_batch_size=4, #train_batch_size,
    per_device_eval_batch_size=4, #self.cfg.eval_batch_size,
    warmup_steps=100,
    weight_decay=0.01,
    # dataloader_drop_last=True,
    disable_tqdm=False,
    push_to_hub=False
)

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_validation,
        data_collator=data_collator,
    )

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
trainer.train() # resume_from_checkpoint=True
trainer.save_model()

Step,Training Loss,Validation Loss
1061,2.075300,1.698496
2122,1.396000,1.359577
3183,1.130300,1.246507


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].


# Load the model and test it on the test dataset

In [3]:
torch.cuda.empty_cache()

In [4]:
input_file = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/mobilerec/splits/test.jsonl"
df_recommender_test = pd.read_json(input_file, lines=True)
for _, row in df_recommender_test.iterrows():
    row["recommended_app"]["app_name"] = row["recommended_app"]["app_name"].lower()

In [5]:
apps_training_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/dataset/mobilerec/mobile_df.csv"

all_apps = []
with open(apps_training_path, 'r') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        all_apps.append(row["app_name"].lower())
        
all_apps = list(set(all_apps))

In [6]:
all_apps

['donut maker cooking games',
 'albion online',
 'transport it! 3d - tycoon manager',
 'usa dating',
 'grifols donor hub',
 'soccer live scores - sofascore',
 'block! hexa puzzle™',
 'twerk race 3d — running game',
 'civilization vi - build a city | strategy 4x game',
 'go launcher s – 3d theme, wallpaper & sticker',
 'garden pets: match-3 dog & cat home decorate',
 'guns at dawn: shooter pvp game',
 'paypal - send, shop, manage',
 'mohre',
 'clover go g1-point of sale',
 'car launcher pro',
 'vector 2 premium',
 'ddtank mobile',
 'destroy watch face',
 'hero factory - idle tycoon',
 'sutter health my health online',
 'ancient life 古代人生',
 'merge meadow - cute animal collector!',
 'farm bay',
 'lily city: building metropolis',
 'basemap: hunting maps, offline nav/gps & weather',
 'support for coparents',
 'piyolog: newborn baby tracker',
 'mindbody: home workout & fitness app',
 'chuchel',
 'pro feel golf - sports simulation',
 "baldur's gate ii: enhanced ed.",
 'univision now: univisi

In [7]:
def candidate_creator(row):
    np.random.seed(row.name)
    selected_values = np.random.choice(np.setdiff1d(all_apps, [row["recommended_app"]["app_name"]]), 24, replace=False) #  
    random_position = np.random.randint(0, len(selected_values) + 1)
    
    return np.insert(selected_values, random_position, row["recommended_app"]["app_name"])

df_recommender_test['candidate'] = df_recommender_test.apply(lambda row: candidate_creator(row), axis=1)

In [8]:
prompt_test = []
recommend_test = []
candidate_books = []
true_candidate_indexes = []
not_founds = 0
for _, row in df_recommender_test.iterrows():
    candidates = []
    for index, candidate_book in enumerate(row["candidate"].tolist()):
        candidates.append(candidate_book)
        if candidate_book == row["recommended_app"]["app_name"]:
            true_candidate_index = index
    prompt = ""
    
    found = False
    recommended = row["recommended_app"]["app_name"]
    
    for index, turn in enumerate(row["turns"]):
        computer = turn["COMPUTER"]
        
        if fuzz.partial_ratio(recommended, computer.lower()) >= 95:
            prompt += "computer: I would recommend the "
            prompt_test.append(prompt)
            recommend_test.append(recommended)
            candidate_books.append(candidates)
            true_candidate_indexes.append(true_candidate_index)
            found = True
            break
        else:
            prompt += "computer: "+ computer + "\n"
        
        if "HUMAN" in turn:
            human = turn["HUMAN"]
            prompt += "human: " + human + "\n"
    
    if not found:
        not_founds += 1

print(f"Could not find {not_founds}")
print(f"Number of prompt: {len(prompt_test)}")
print(f"Number of generations: {len(recommend_test)}")
print(f"Number of candidate apps: {len(candidate_books)}")
print(f"Number of true candidate indexes: {len(true_candidate_indexes)}")

Could not find 16
Number of prompt: 1831
Number of generations: 1831
Number of candidate apps: 1831
Number of true candidate indexes: 1831


In [10]:
candidate_books[10]

['my time at portia',
 'solitaire card games, klondike',
 'toca life: school',
 'teppen',
 'usa dating',
 'safe surfer: block porn & apps',
 'pdf scanner app - scan to pdf',
 'death worm™ deluxe',
 'learn spanish - español',
 'instagram',
 'nysoh mobile upload',
 'syncup tracker',
 'idle guardians: never die',
 'slots vegas magic casino 777',
 'candy camera - photo editor',
 'showtime viewer from zoho',
 'kink dating life: bdsm dating, fetish & kinky life',
 'farkle online - 10000 dice game',
 'teladoc | telehealth & therapy',
 'creative destruction',
 'geico mobile - car insurance',
 'baby panda care',
 'kawaiinihongo: learn japanese',
 'yokai tamer-new contents',
 'seven - 7 minute workout']

In [21]:
model = AutoModelForSeq2SeqLM.from_pretrained(pretrained_model_name_or_path = "/u/spa-d4/grad/mfe261/Projects/MobileConvRec/models/new_models/mobilerec/T5_recommender")
model.eval()
model = model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base", additional_special_tokens=["computer:", "human:"])
model.resize_token_embeddings(len(tokenizer))
IGNORE_INDEX = -100

/mounts/u-spa-d4/grad/mfe261/Projects/MobileConvRec/envs/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [22]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]

def evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70):
  prompt_batches = list(chunk(prompt_test, batch_size))
  generation_batches = list(chunk(recommend_test, batch_size))

  correctly_predicted = []
  for prompt_batch, generation_batch in tqdm(zip(prompt_batches, generation_batches), total = len(generation_batches)):

    inputs = tokenizer(prompt_batch, max_length=1024, truncation=True, padding="max_length", return_tensors="pt") 

    generations_predicted = model.generate(input_ids=inputs["input_ids"].to('cuda'), attention_mask=inputs["attention_mask"].to('cuda'),
                            max_new_tokens=32,
                            num_beams=8,
                            eos_token_id=tokenizer.eos_token_id,
                            pad_token_id=tokenizer.pad_token_id,
                            bos_token_id=tokenizer.bos_token_id) # length_penalty=0.8, Set length_penalty to values < 1.0 in order to encourage the model to generate shorter sequences, to a value > 1.0 in order to encourage the model to produce longer sequences.

    decoded_generations = [tokenizer.decode(generation, skip_special_tokens=True, clean_up_tokenization_spaces=True) for generation in generations_predicted]
    generation_batch = [generation for generation in generation_batch]
    
    correctly_predicted.extend([1 if fuzz.ratio(predicted, ground_truth) > threshold else 0 for predicted, ground_truth in zip(decoded_generations, generation_batch)])

  return correctly_predicted

In [23]:
correctly_predicted = evaluate_recommender(prompt_test, recommend_test, model, tokenizer, batch_size=8, threshold=70)
success_rate = sum(correctly_predicted) / len(correctly_predicted)
print("success_rate: ", success_rate)

  0%|          | 0/229 [00:00<?, ?it/s]

100%|██████████| 229/229 [03:25<00:00,  1.11it/s]

success_rate:  0.3413435281267067


In [24]:
def chunk(list_of_elements, batch_size): # using this chunk function, we can split our data to multiple batches
  for i in range(0, len(list_of_elements), batch_size):
    yield list_of_elements[i:i+batch_size]
    
def convert_to_sublists(numbers, sublist_size):
    return [numbers[i:i+sublist_size] for i in range(0, len(numbers), sublist_size)]

def recommender_rank(prompts, candidate_apps, model, tokenizer, batch_size=8):
  model.eval()
  encoder_max_length = 1024
  decoder_max_length = 32
  prompts_tokenized = tokenizer(prompts, max_length=encoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
  
  input_ids_decoder = []
  attention_mask_decoder = []
  input_ids_encoder = []
  attention_mask_encoder  = []
  for index, candidate_app_elements in enumerate(candidate_apps):
    candidate_app_elements = [tokenizer.pad_token+element for element in candidate_app_elements] # adding pad token to the beginning of each candidate app
    candidate_apps_tokenized = tokenizer(candidate_app_elements, max_length=decoder_max_length, truncation=True, padding="max_length", return_tensors="pt")
    for candidate_app_index in range(len(candidate_app_elements)):
      input_ids_decoder.append(candidate_apps_tokenized["input_ids"][candidate_app_index])
      attention_mask_decoder.append(candidate_apps_tokenized["attention_mask"][candidate_app_index])
      input_ids_encoder.append(prompts_tokenized["input_ids"][index])
      attention_mask_encoder.append(prompts_tokenized["attention_mask"][index])
  
  input_ids_encoder_batches = list(chunk(input_ids_encoder, batch_size))
  attention_mask_encoder_batches = list(chunk(attention_mask_encoder, batch_size))
  input_ids_decoder_batches = list(chunk(input_ids_decoder, batch_size))
  attention_mask_decoder_batches = list(chunk(attention_mask_decoder, batch_size))
  

  scores = []
  for input_ids_encoder_batch, attention_mask_encoder_batch, input_ids_decoder_batch, attention_mask_decoder_batch in tqdm(zip(input_ids_encoder_batches, attention_mask_encoder_batches, input_ids_decoder_batches, attention_mask_decoder_batches), total = len(input_ids_encoder_batches)):
    decoder_input_ids = torch.stack(input_ids_decoder_batch).to("cuda")
    decoder_attention_mask = torch.stack(attention_mask_decoder_batch).to("cuda")
    input_ids = torch.stack(input_ids_encoder_batch).to("cuda")
    attention_mask = torch.stack(attention_mask_encoder_batch).to("cuda")
    with torch.no_grad():
      model_output = model(decoder_input_ids=decoder_input_ids, decoder_attention_mask=decoder_attention_mask, 
                           input_ids=input_ids, attention_mask=attention_mask)
    
    logprobs = F.log_softmax(model_output["logits"], dim=-1)[:, :-1, :] # remove the eos token
    output_tokens = decoder_input_ids[:, 1:] # remove the bos token
        
    tokens_logprobs = torch.gather(logprobs, 2, output_tokens[:, :, None]).squeeze(-1).to(torch.float32)
        
    mask = torch.ones(tokens_logprobs.shape, dtype=torch.bool, device="cuda")
    for i, _output in enumerate(output_tokens):
      for j, _token in enumerate(_output):
        if _token == tokenizer.pad_token_id:
          mask[i, j] = False
              
    score = (tokens_logprobs * mask).sum(-1) / mask.sum(-1)
    scores.extend(score.to('cpu').tolist())
    
  # batch_input_representations = torch.cat(batch_input_representations)
  
  scores = convert_to_sublists(scores, len(candidate_apps[0]))
  
  return scores

In [25]:
scores = recommender_rank(prompt_test, candidate_books, model, tokenizer, batch_size=8)

100%|██████████| 5722/5722 [21:35<00:00,  4.42it/s]


Sampled Candidates

In [26]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

[np.float64(0.7089022392135446),
 np.float64(0.7711632987438558),
 np.float64(0.803386127799017),
 np.float64(0.8339705079191698),
 np.float64(0.8492626979792464),
 np.float64(0.8667394866193336),
 np.float64(0.880393227744402),
 np.float64(0.8918623702894594),
 np.float64(0.9044238121245222),
 np.float64(0.914800655379574)]

In [27]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [28]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]

[np.float64(0.7089022392135446),
 np.float64(0.7481845941601017),
 np.float64(0.7642960086876822),
 np.float64(0.777467984248638),
 np.float64(0.7833838109021427),
 np.float64(0.7896091686233098),
 np.float64(0.7941604156649991),
 np.float64(0.7977785273048136),
 np.float64(0.8015598980859558),
 np.float64(0.8045594784792054)]

Similar Candidates

In [ ]:
[top_k_accuracy_score(true_candidate_indexes, scores, k=k) for k in range(1, 11)]

In [44]:
true_relevance = [[1 if item == index else 0 for item in range(len(candidate_books[0]))] for index in true_candidate_indexes]

In [ ]:
[ndcg_score(true_relevance, scores, k=k) for k in range(1, 11)]